In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/chinese task/CLUENER2020')

In [ ]:
#安装
!pip install transformers datasets

In [ ]:
import os
import json
import logging
import numpy as np
import pandas as pd
import config

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [ ]:
#加载处理完的npz数据集
#不加allow_pickle=True会报错Object arrays cannot be loaded when allow_pickle=False，numpy新版本中默认为False。
train_data=np.load('./data/train.npz',allow_pickle=True)
val_data=np.load('./data/dev.npz',allow_pickle=True)
test_data=np.load('./data/test.npz',allow_pickle=True)

test_data.files

['words', 'labels']

In [ ]:
#转换为dataframe格式
import pandas as pd
#补个随机frac
train_df=pd.concat([pd.DataFrame(train_data['words'],columns=['words']),
          pd.DataFrame(train_data['labels'],columns=['labels'])],axis=1)

val_df=pd.concat([pd.DataFrame(val_data['words'],columns=['words']),
          pd.DataFrame(val_data['labels'],columns=['labels'])],axis=1)

test_df=pd.concat([pd.DataFrame(test_data['words'],columns=['words']),
          pd.DataFrame(test_data['labels'],columns=['labels'])],axis=1)

#将训练验证集的BIOS标签转换为数字索引，此时word和labels已经对齐了
def trans(labels):
  labels=list(labels)
  nums=[]
  for label in labels:
    nums.append(config.label2id[label])
  return nums
    
train_df['labels']=train_df['labels'].map(lambda x: trans(x))
val_df['labels']=val_df['labels'].map(lambda x: trans(x))

test_df['labels']=test_df['labels'].map(lambda x: trans(x))
val_df

,words,labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ..."


## pandas数据装入datasets进行解码，之后方便直接pad labels。
## labels在后面装入dataloader的时候，不处理的话长度不一致，处理的话整理函数太麻烦。

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer
#这里一定要选AutoTokenizer，如果是BertTokenizer，会提示bertbase没有word_ids方法。结果没用到
trains_ds=Dataset.from_pandas(train_df)
val_ds=Dataset.from_pandas(val_df)
test_ds=Dataset.from_pandas(test_df)

tokenizer=AutoTokenizer.from_pretrained(config.roberta_model,do_lower_case=True)

#tokenized_inputs=tokenizer(trains_ds["words"],padding=True,truncation=True,is_split_into_words=True)为啥这种是错的
tokenized_trains_ds=trains_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)
tokenized_val_ds=val_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)
tokenized_test_ds=test_ds.map(lambda examples:tokenizer(examples['words'],is_split_into_words=True,truncation=True,padding=True),batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
#在编码之后的datasets里面操作，得到的结果无法写入datasets，所以只好写到pandas文件里。
#将labels填充到和input_ids一样长（最长句子52，所以其实全部都填充到52）

def padding(data):
  pad_labels=[]
  for ds in data:
    labels=ds['labels'] 
    mask=ds['attention_mask']

    pad_length=len(mask)
    label_length=len(labels)
    
    label_ids=[-100]+labels+[-100]*(pad_length-label_length-1)
    pad_labels.append(label_ids)
  return pad_labels
#tokenized_trains_ds["pad_labels"]=pad_labels# Column 2 named labels expected length 10748 but got length 1000
train_df['pad_labels']=padding(tokenized_trains_ds)#
val_df['pad_labels']=padding(tokenized_val_ds)
test_df['pad_labels']=padding(tokenized_test_ds)
test_df

,words,labels,pad_labels
0,"[四, 川, 敦, 煌, 学, ”, 。, 近, 年, 来, ，, 丹, 棱, 县, 等, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[尼, 日, 利, 亚, 海, 军, 发, 言, 人, 当, 天, 在, 阿, 布, 贾, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[销, 售, 冠, 军, ：, 辐, 射, 3, -, B, e, t, h, e, s, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[所, 以, 大, 多, 数, 人, 都, 是, 从, 巴, 厘, 岛, 南, 部, 开, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[备, 受, 瞩, 目, 的, 动, 作, 及, 冒, 险, 类, 大, 作, 《, 迷, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
1340,"[貌, 似, 主, 队, 优, 势, 不, 小, 。, 不, 过, 欧, 平, 客, 胜, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1341,"[当, 年, 赵, 国, 都, 城, 邯, 郸, 被, 秦, 军, 包, 围, ，, 赵, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1342,"[关, 于, 张, 衡, 地, 动, 仪, 的, 历, 史, 证, 据, ，, 冯, 锐, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1343,"[一, 直, 面, 带, 微, 笑, ，, 任, 由, 化, 妆, 师, 为, 她, 化, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
%pylab inline
#最大句子长度50
train_df['text_len'] = train_df['words'].apply(lambda x: len(x))
print(train_df['text_len'].describe())

Populating the interactive namespace from numpy and matplotlib
count    10748.000000
mean        37.380350
std         10.709827
min          2.000000
25%         32.000000
50%         41.000000
75%         46.000000
max         50.000000
Name: text_len, dtype: float64


In [ ]:
#每个句子都被pad到52的长度
train_df['label_len'] = train_df['pad_labels'].apply(lambda x: len(x))
print(train_df['label_len'].describe())

count    10748.0
mean        52.0
std          0.0
min         52.0
25%         52.0
50%         52.0
75%         52.0
max         52.0
Name: label_len, dtype: float64


In [ ]:
val_df

,words,labels,pad_labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-100, 7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-100, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ...","[-100, 4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0...","[-100, 0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-100, 0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[-100, 6, 16, 16, 16, 16, 16, 16, 16, 16, 16, ..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0...","[-100, 0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15,..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, ..."


In [ ]:
print(type(test_df['labels'][0][0]))

<class 'int'>


In [ ]:
batch_size=32

#划分训练验证集
from sklearn.model_selection import train_test_split
from datasets import Dataset
from torch.nn.utils.rnn import pad_sequence


train_data,val_data,train_label,val_label=train_test_split(
    train_df['words'].iloc[:], 
    train_df['pad_labels'].iloc[:],
    test_size=0.15,shuffle=True)

test_data,test_label=(test_df['words'].iloc[:],test_df['pad_labels'].iloc[:])
validation_data,validation_label=(val_df['words'].iloc[:],val_df['pad_labels'].iloc[:])
#stratify=train_df['label'].iloc[:]报错:The least populated class in y has only 1 member,which is too few.
#The minimum number of groups for any class cannot be less than 2.估计是样本太少，分层抽取不可行。

#数据预处理

tokenizer=AutoTokenizer.from_pretrained(config.roberta_model,do_lower_case=True)
train_encoding=tokenizer(list(train_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#训练集中划分的训练集
val_encoding=tokenizer(list(val_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#训练集中划分的验证集
test_encoding=tokenizer(list(test_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#测试集

validation_econding=tokenizer(list(validation_data),is_split_into_words=True,truncation=True,padding=True,return_tensors='pt')#原本的验证集


In [ ]:
#加载到datalodar并预处理
#数据集读取

from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch
class XFeiDataset(Dataset):
  def __init__(self,encodings,labels):
    self.encodings=encodings
    self.labels=labels
  
  # 读取单个样本
  def __getitem__(self,idx):
    item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['pad_labels']=torch.tensor((self.labels[idx]))
    item['mask']=(item['pad_labels']!=-100)
    return item
  
  def __len__(self):
    return len(self.labels)

#def collate_fn

train_dataset=XFeiDataset(train_encoding,list(train_label))
val_dataset=XFeiDataset(val_encoding,list(val_label))
test_dataset=XFeiDataset(test_encoding,list(test_label))
validation_dataset=XFeiDataset(validation_econding,list(validation_label))

from torch.utils.data import Dataset,DataLoader,TensorDataset

train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)#test数据不能shuffle啊，真坑死我了
validation_loader=DataLoader(validation_dataset,batch_size=batch_size,shuffle=False)#test数据不能shuffle啊，真坑死我了

In [ ]:
from transformers import BertModel
from torch.nn.utils.rnn import pad_sequence
#初始化bert模型
from transformers import BertConfig
import torch.nn as nn
from torch.nn import LSTM



num_labels=31
dropout=0.1

class Bert_LSTM(nn.Module):
  def __init__(self):
    super(Bert_LSTM,self).__init__()
    self.num_labels=num_labels
    self.dropout=nn.Dropout(dropout)
    self.bert=BertModel.from_pretrained(config.roberta_model)
    for param in self.bert.parameters():
      param.requires_grad=True
    self.classifier=nn.Linear(1024,self.num_labels)
    #self.crf=CRF(num_labels,batch_first=True)
    
    self.bilstm=nn.LSTM(
        input_size=1024, 
        hidden_size=512, 
        batch_first=True,
        num_layers=2,
        dropout=0.5,  
        bidirectional=True)

  def forward(self,batch_seqs,batch_seq_masks,batch_seq_segments):

    output=self.bert(input_ids=batch_seqs,attention_mask=batch_seq_masks,token_type_ids=batch_seq_segments)
    #pooler_output=output.pooler_output
    last_hidden_state=output.last_hidden_state

    if model.train():
      last_hidden_state=self.dropout(last_hidden_state)
    #只有这种写法不会报错，如果是sequence_output,pooler_output=self.bert(**kwags)这种，sequence_output会报错str没有xxx属性。
    #貌似是bert输出有很多，直接用output.last_hidden_state来调用结果（估计是版本问题，坑），关键是输出要打印出来
    

    lstm_output,(hn,cn)=self.bilstm(last_hidden_state)
    #output为输出序列的隐藏层，hn为最后一个时刻的隐藏层，cn为最后一个时刻的隐藏细胞
    if model.train():
      lstm_output=self.dropout(lstm_output)

    # 得到判别值
    logits=self.classifier(lstm_output)
    
    
    return logits   

In [ ]:
#加载模型
model=Bert_LSTM()
#model.load_state_dict(torch.load("best_bert_model_3epoch"))
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
#定义优化器
epoch=10
lr=3e-5

from transformers import AdamW,get_scheduler

train_steps_per_epoch=len(train_loader)
num_training_steps=train_steps_per_epoch*epoch

#定义各模块参数
bert_parameters=list(model.bert.named_parameters())
lstm_parameters=list(model.bilstm.named_parameters())
classifier_parameters=list(model.classifier.named_parameters())
no_decay=['bias','LayerNorm.weight']

#bert模型、lstm模型、nn.linear的学习率分离，后两个是bert的3倍
optimizer_grouped_parameters=[
    {'params':[p for n,p in bert_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.01},
    {'params':[p for n,p in bert_parameters if any(nd in n for nd in no_decay)],
      'lr':lr,'weight_decay':0.0},
    {'params':[p for n,p in lstm_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in lstm_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay': 0.0},
    {'params':[p for n,p in classifier_parameters if not any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.01},
    {'params':[p for n,p in classifier_parameters if any(nd in n for nd in no_decay)],
      'lr':lr*3,'weight_decay':0.0}]

optimizer=AdamW(optimizer_grouped_parameters,lr=lr,eps=1e-8)

lr_scheduler=get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps)

In [ ]:
#编写训练和验证循环
import time
import numpy as np
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score
from torch.nn import functional as F
from torchcrf import CRF
#加载进度条
from tqdm.auto import tqdm

num_training_steps=train_steps_per_epoch*epoch

progress_bar=tqdm(range(num_training_steps))

def train_and_eval(epoch):
  best_acc=0.0
  num_labels=31
  criterion=nn.CrossEntropyLoss()
  for i in range(epoch):
    """训练模型"""
    start=time.time()
    model.train()
    print("***** Running training epoch {} *****".format(i+1))
    train_loss_sum=0.0
    for idx,batch in enumerate(train_loader):
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      pad_labels=batch['pad_labels'].to(device)
      mask=batch['mask'].to(device)


      #计算输出和loss
      logits=model(input_ids,attention_mask,token_type_ids)

      loss=criterion(logits[mask],pad_labels[mask])
      loss.backward()

      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()  
      progress_bar.update(1)

      train_loss_sum+=loss.item()
      if (idx+1)%(len(train_loader)//5)==0: # 只打印五次结果
        print("Epoch {:03d} | Step {:04d}/{:04d} | Loss {:.4f} | Time {:.4f} | Learning rate = {} \n".format(
                  i+1,idx+1,len(train_loader),train_loss_sum/(idx+1),time.time()-start,optimizer.state_dict()['param_groups'][0]['lr']))
      
      #验证模型
    model.eval()
    total=0#每个batch要预测的token总数
    acc=0#每个batch的acc
    total_eval_accuracy=0
    total_eval_loss=0
    
    for batch in val_loader:
      with torch.no_grad():#只有这一块是不需要求导的
      
        input_ids=batch['input_ids'].to(device)
        attention_mask=batch['attention_mask'].to(device)
        token_type_ids=batch['token_type_ids'].to(device)
        pad_labels=batch['pad_labels'].to(device)
        mask=batch['mask'].to(device)
        logits=model(input_ids,attention_mask,token_type_ids)


      #logits[mask]从句子矩阵变被拉平，且只含有真实token的logtis。和bertfortoken分类任务头的view效果是一样的。
      loss=criterion(logits[mask],pad_labels[mask])#只计算没有mask的部分单词的loss和准确率
      
      total_eval_loss+=loss.item()

      acc+=(logits.argmax(dim=-1)[mask]==pad_labels[mask]).sum().item()#只计算没有mask的单词的准确率,mask在外面似乎accs0.93不准。
      total+=mask.sum().item()
    total_eval_accuracy=acc/total

    #avg_val_accuracy=total_eval_accuracy/len(val_loader)
    if total_eval_accuracy>best_acc:
      best_acc=total_eval_accuracy
      torch.save(model.state_dict(),"bert_lstm_softmax_model")
    
    print("val_accuracy:%.4f" % (total_eval_accuracy))
    print("Average val loss: %.4f"%(total_eval_loss))
    print("time costed={}s \n".format(round(time.time()-start,5)))
    print("-------------------------------")

  0%|          | 0/2860 [00:00<?, ?it/s]

## 对比bert的token分类任务头

In [ ]:
train_and_eval(epoch)

***** Running training epoch 1 *****


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


Epoch 001 | Step 0057/0286 | Loss 1.5436 | Time 143.0378 | Learning rate = 2.94020979020979e-05 

Epoch 001 | Step 0114/0286 | Loss 1.4414 | Time 285.9572 | Learning rate = 2.8804195804195805e-05 

Epoch 001 | Step 0171/0286 | Loss 1.2982 | Time 428.8080 | Learning rate = 2.820629370629371e-05 

Epoch 001 | Step 0228/0286 | Loss 1.1858 | Time 571.7140 | Learning rate = 2.760839160839161e-05 

Epoch 001 | Step 0285/0286 | Loss 1.0738 | Time 714.6863 | Learning rate = 2.701048951048951e-05 

val_accuracy:0.8461
Average val loss: 26.1596
time costed=765.61637s 

-------------------------------
***** Running training epoch 2 *****
Epoch 002 | Step 0057/0286 | Loss 0.4251 | Time 142.9354 | Learning rate = 2.6402097902097903e-05 

Epoch 002 | Step 0114/0286 | Loss 0.3791 | Time 285.8664 | Learning rate = 2.5804195804195803e-05 

Epoch 002 | Step 0171/0286 | Loss 0.3487 | Time 428.8010 | Learning rate = 2.5206293706293707e-05 

Epoch 002 | Step 0228/0286 | Loss 0.3313 | Time 571.7066 | Learni

KeyboardInterrupt: ignored

In [ ]:
#用trainer预测验证集结果并保存
#torch.save(model.state_dict(),"best_lstm_whole_4epoch")
model.load_state_dict(torch.load("after_test_bert_lstm_softmax_model"))
model.to(device)

In [ ]:
model.eval()
criterion=nn.NLLLoss()#不带softmax的损失函数
total=0#每个batch要预测的token总数
acc=0#每个batch的acc
total_eval_accuracy=0
total_eval_loss=0
val_pred = []

for batch in validation_loader:  
  with torch.no_grad():#只有这一块是不需要求导的
  
    input_ids=batch['input_ids'].to(device)
    attention_mask=batch['attention_mask'].to(device)
    token_type_ids=batch['token_type_ids'].to(device)
    pad_labels=batch['pad_labels'].to(device)
    mask=batch['mask'].to(device)
    logits=model(input_ids,attention_mask,token_type_ids)

  #logits[mask]从句子矩阵变被拉平，且只含有真实token的logtis。和bertfortoken分类任务头的view效果是一样的。
  loss=criterion(logits[mask],pad_labels[mask])#只计算没有mask的部分单词的loss和准确率
  total_eval_loss+=loss.item()

  acc+=(logits.argmax(dim=-1)==pad_labels)[mask].sum().item()#只计算没有mask的单词的准确率
  total+=mask.sum().item()
  total_eval_accuracy=acc/total
print("val_accuracy:%.4f" % (total_eval_accuracy))
print("Average val loss: %.4f"%(total_eval_loss))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


val_accuracy:0.9341
Average val loss: 11.3264


In [ ]:
#编写predict函数
def predict(model,data_loader):#参数名为data时加载训练好的模型来预测报错，原模型不报错
  model.eval()
  test_pred = []
  for batch in data_loader:
    with torch.no_grad():
      input_ids=batch['input_ids'].to(device)
      attention_mask=batch['attention_mask'].to(device)
      token_type_ids=batch['token_type_ids'].to(device)
      mask=batch['mask'].to(device)
      logits=model(input_ids,attention_mask,token_type_ids)
      pad_logits=logits[mask]

      y_pred=torch.argmax(logits,dim=-1).detach().cpu().numpy()#为啥最后拉平的又变回矩阵了，看不懂啊
      test_pred.extend(y_pred)
      
  return test_pred

## 输入一个validation_loader的数据，打印出来看看为啥变回52长度的矩阵（应该是模型只能输入输出等长的矩阵）

In [ ]:
ls={}
for data in train_loader:
  ls={key:val for key,val in data.items()}
  break
ls

from torchcrf import CRF
crf=CRF(31,batch_first=True)
input_ids=ls['input_ids'].to(device)
attention_mask=ls['attention_mask'].to(device)
token_type_ids=ls['token_type_ids'].to(device)
pad_labels=ls['pad_labels'].to(device)
mask=ls['mask'].to(device)
logits=model(input_ids,attention_mask,token_type_ids)
print(mask.shape,logits.shape,pad_labels.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


torch.Size([16, 51]) torch.Size([16, 52, 31]) torch.Size([16, 51])


In [ ]:
crf.to(device)
logits=logits[:,1:,:].to(device)


#loss=crf(logits,pad_labels,mask)
#pad_logits=logits[mask]

#y_pred=torch.argmax(logits,dim=-1).detach().cpu().numpy()
#test_pred.extend(y_pred)
print(logits.shape)
#print(logits,loss)

torch.Size([16, 51, 31])


## mask真假矩阵来过滤，得到的真实token的结果矩阵是每行长度不一致，所以被拉平了。loss计算的是拉平的有效词的loss。

##预测时不能mask过滤，否则报错。

In [ ]:
loss=crf(logits,pad_labels,mask)
loss

In [ ]:
pad_logits=logits[mask]#多个句子的token矩阵，经过mask矩阵被拉平。
y_pred=torch.argmax(pad_logits,dim=-1)
print(pad_logits,pad_logits.shape)
print('--------------------------------------------------------------------------------------------------------')
print(y_pred,y_pred.shape)

tensor([[-6.8415e+00, -7.9797e+00, -6.9759e+00,  ..., -1.2822e+01,
         -1.2866e+01, -1.2334e+01],
        [-6.4157e+00, -1.2603e+01, -1.1605e+01,  ..., -1.2643e+01,
         -1.2956e+01, -1.2452e+01],
        [-5.5986e+00, -1.2277e+01, -1.3045e+01,  ..., -1.3453e+01,
         -1.3545e+01, -1.3535e+01],
        ...,
        [-6.8185e-05, -1.1929e+01, -1.4428e+01,  ..., -1.7257e+01,
         -1.6416e+01, -1.7380e+01],
        [-8.2966e-05, -1.1916e+01, -1.4230e+01,  ..., -1.6780e+01,
         -1.6141e+01, -1.7019e+01],
        [-6.4609e-05, -1.2297e+01, -1.4409e+01,  ..., -1.7191e+01,
         -1.6353e+01, -1.7541e+01]], device='cuda:0', grad_fn=<IndexBackward0>) torch.Size([1219, 31])
--------------------------------------------------------------------------------------------------------
tensor([ 7, 17, 17,  ...,  0,  0,  0], device='cuda:0') torch.Size([1219])


In [ ]:
import torch
from torch import tensor
a=torch.randn(2,4)
b=tensor([[False,True,True,False],
 [True,False,True,True]])
c=a[b]

print(a,a.shape)
print(c,c.shape)
#mask矩阵可以晒选需要的位置,但是确实被拉平了

tensor([[-0.2196,  0.1262, -0.6929, -1.7824],
        [-0.4014, -0.5301, -0.6155,  0.6116]]) torch.Size([2, 4])
tensor([ 0.1262, -0.6929, -0.4014, -0.6155,  0.6116]) torch.Size([5])


## 用模型预测验证集结果，与原标签对比

In [ ]:
#用trainer预测验证集结果并保存
#torch.save(model.state_dict(),"best_lstm_whole_4epoch")
#model.load_state_dict(torch.load("after_test_bert_lstm_softmax_model"))
#model.to(device)

predictions=predict(model,validation_loader)
val_df['pre_labels']=pd.Series(predictions)
from datasets import Dataset
val_datasets=Dataset.from_pandas(val_df)
#将预测的结果直接加到val_df。如果存入csv读取出来再加入，读取的labels数据就是文本数据，坑了好久才发现。而且还有换行符，醉了。

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


In [ ]:
def unpadding(data):
  unpad_labels=[]
  for ds in data:#直接这样迭代读取报错。pandas数据不能这样读取每一行。datasets是dict格式，可以用datasets['train'][0]这样的方式读取
    pad_labels=ds['pre_labels'] #这里是pre_labels，又他妈写错了
    words=ds['words']

    length=len(words)
    label_ids=pad_labels[1:(length+1)]

    unpad_labels.append(label_ids)
  return unpad_labels
#tokenized_trains_ds["pad_labels"]=pad_labels# Column 2 named labels expected length 10748 but got length 1000
val_df['unpad_labels']=unpadding(val_datasets)
val_df

,words,labels,pad_labels,pre_labels,unpad_labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-100, 7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[-100, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ...","[-100, 4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17...","[0, 4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 1...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0...","[-100, 0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[-100, 0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[-100, 6, 16, 16, 16, 16, 16, 16, 16, 16, 16, ...","[0, 7, 17, 17, 0, 6, 16, 16, 16, 16, 16, 16, 1...","[7, 17, 17, 0, 6, 16, 16, 16, 16, 16, 16, 16, ..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0...","[-100, 0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15,...","[0, 0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20..."


In [ ]:
val_df.drop(columns=(['pad_labels','pre_labels']),inplace=True)
val_df.to_csv('bert_lstm_validation_1113.csv')
val_df

,words,labels,unpad_labels
0,"[彭, 小, 军, 认, 为, ，, 国, 内, 银, 行, 现, 在, 走, 的, 是, ...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
1,"[温, 格, 的, 球, 队, 终, 于, 又, 踢, 了, 一, 场, 经, 典, 的, ...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,"[突, 袭, 黑, 暗, 雅, 典, 娜, 》, 中, R, i, d, d, i, c, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ...","[4, 14, 14, 14, 14, 14, 14, 14, 0, 7, 17, 17, ..."
3,"[郑, 阿, 姨, 就, 赶, 到, 文, 汇, 路, 排, 队, 拿, 钱, ，, 希, ...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 1, 11, 11, 0, 0, 0, 0, 0, 0..."
4,"[我, 想, 站, 在, 雪, 山, 脚, 下, 你, 会, 被, 那, 巍, 峨, 的, ...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
...,...,...,...
1338,"[在, 这, 个, 非, 常, 喜, 庆, 的, 日, 子, 里, ，, 我, 们, 首, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1339,"[姜, 哲, 中, ：, 公, 共, 之, 敌, 1, -, 1, 》, 、, 《, 神, ...","[6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16...","[7, 17, 17, 0, 6, 16, 16, 16, 16, 16, 16, 16, ..."
1340,"[目, 前, ，, 日, 本, 松, 山, 海, 上, 保, 安, 部, 正, 在, 就, ...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0...","[0, 0, 0, 5, 15, 15, 15, 15, 15, 15, 15, 15, 0..."
1341,"[也, 就, 是, 说, 英, 国, 人, 在, 世, 博, 会, 上, 的, 英, 国, ...","[0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20..."


In [ ]:
val_df.columns

Index(['words', 'labels', 'unpad_labels'], dtype='object')

In [ ]:
for i in tokenized_val_ds:#这里写val_ds读出来的就是'word'。pandas列间运算要另外写。
  print(i)
  break

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 2510, 2207, 1092, 6371, 711, 8024, 1744, 1079, 7213, 6121, 4385, 1762, 6624, 4638, 3221, 1378, 3968, 4638, 1355, 1305, 3563, 2466, 8024, 1044, 6858, 6814, 6651, 7716, 1750, 1765, 1086, 1762, 1750, 4638, 1765, 7027, 7481, 6848, 2885, 2145, 2787, 8024, 102, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [7, 17, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'words': ['彭', '小', '军', '认', '为', '，', '国', '内', '银', '行', '现', '在', '走', '的', '是', '台', '湾', '的', '发', '卡', '模', '式', '，', '先', '通', '过', '跑', '马', '圈', '地', '再', '在', '圈', '的', '地', '里', '面', '选', '择', '

In [ ]:
#预测测试集结果并保存
#torch.save(model.state_dict(),"best_lstm_whole_4epoch")
#model.load_state_dict(torch.load("after_test_bert_lstm_softmax_model"))
#model.to(device)
"""
test_predictions=predict(model,test_loader)
test_df['pre_labels']=pd.Series(test_predictions)
test_datasets=Dataset.from_pandas(test_df)
test_df['unpad_labels']=unpadding(test_datasets)
"""
test_df.drop(columns=(['labels','pad_labels','pre_labels']),inplace=True)
test_df

,words,unpad_labels
0,"[四, 川, 敦, 煌, 学, ”, 。, 近, 年, 来, ，, 丹, 棱, 县, 等, ...","[1, 11, 11, 18, 0, 0, 0, 0, 0, 0, 0, 1, 11, 11..."
1,"[尼, 日, 利, 亚, 海, 军, 发, 言, 人, 当, 天, 在, 阿, 布, 贾, ...","[5, 15, 15, 15, 15, 15, 0, 0, 0, 0, 0, 0, 1, 1..."
2,"[销, 售, 冠, 军, ：, 辐, 射, 3, -, B, e, t, h, e, s, ...","[0, 0, 0, 0, 0, 4, 14, 14, 0, 0, 0, 14, 14, 14..."
3,"[所, 以, 大, 多, 数, 人, 都, 是, 从, 巴, 厘, 岛, 南, 部, 开, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 20, 20, 0, 0, ..."
4,"[备, 受, 瞩, 目, 的, 动, 作, 及, 冒, 险, 类, 大, 作, 《, 迷, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 14,..."
...,...,...
1340,"[貌, 似, 主, 队, 优, 势, 不, 小, 。, 不, 过, 欧, 平, 客, 胜, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1341,"[当, 年, 赵, 国, 都, 城, 邯, 郸, 被, 秦, 军, 包, 围, ，, 赵, ...","[0, 0, 0, 0, 0, 0, 1, 11, 0, 0, 0, 0, 0, 0, 0,..."
1342,"[关, 于, 张, 衡, 地, 动, 仪, 的, 历, 史, 证, 据, ，, 冯, 锐, ...","[0, 0, 7, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 17..."
1343,"[一, 直, 面, 带, 微, 笑, ，, 任, 由, 化, 妆, 师, 为, 她, 化, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 19, 19, 0, 0, 0..."


## 想写个返回元组的dataloader，有时间在慢慢试把。还有整理函数里面pad labels数据。